## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Poplar Bluff,US,36.7570,-90.3929,92.34,46,0,1.01,clear sky
1,1,Barrow,US,71.2906,-156.7887,60.82,67,75,4.61,broken clouds
2,2,Cap Malheureux,MU,-19.9842,57.6142,70.47,66,92,8.99,overcast clouds
3,3,Rikitea,PF,-23.1203,-134.9692,72.91,77,13,3.98,few clouds
4,4,Busselton,AU,-33.6500,115.3333,54.82,63,96,2.80,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 60


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Busselton,AU,-33.6500,115.3333,54.82,63,96,2.80,overcast clouds
10,10,Tuktoyaktuk,CA,69.4541,-133.0374,50.00,93,100,4.61,mist
14,14,Hermanus,ZA,-34.4187,19.2345,50.81,69,56,7.20,broken clouds
43,43,Narsaq,GL,60.9167,-46.0500,57.42,80,9,4.38,clear sky
45,45,Iqaluit,CA,63.7506,-68.5145,53.33,66,20,3.44,few clouds
47,47,Esperance,AU,-33.8667,121.9000,54.66,77,81,8.72,light rain
48,48,Nikolskoye,RU,59.7035,30.7861,58.91,98,31,2.01,scattered clouds
55,55,Husavik,IS,66.0449,-17.3389,53.37,94,100,1.99,overcast clouds
58,58,Ahipara,NZ,-35.1667,173.1667,56.98,91,100,8.75,overcast clouds
60,60,Port Elizabeth,ZA,-33.9180,25.5701,55.69,90,20,11.50,few clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum()

City_ID        0
City           0
Country        6
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how="any")
clean_df.isna().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Busselton,AU,54.82,overcast clouds,-33.6500,115.3333,
10,Tuktoyaktuk,CA,50.00,mist,69.4541,-133.0374,
14,Hermanus,ZA,50.81,broken clouds,-34.4187,19.2345,
43,Narsaq,GL,57.42,clear sky,60.9167,-46.0500,
45,Iqaluit,CA,53.33,few clouds,63.7506,-68.5145,
47,Esperance,AU,54.66,light rain,-33.8667,121.9000,
48,Nikolskoye,RU,58.91,scattered clouds,59.7035,30.7861,
55,Husavik,IS,53.37,overcast clouds,66.0449,-17.3389,
58,Ahipara,NZ,56.98,overcast clouds,-35.1667,173.1667,
60,Port Elizabeth,ZA,55.69,few clouds,-33.9180,25.5701,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df[hotel_df['Hotel Name'] == ''].index

Int64Index([107, 111, 143, 241, 250, 260, 307, 334, 342, 358, 423, 452, 520,
            574, 637, 678],
           dtype='int64')

In [26]:
clean_hotel = hotel_df.drop([107, 111, 143, 241, 250, 260, 307, 334, 342, 358, 423, 452, 520,
            574, 637, 678])
clean_hotel

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Busselton,AU,54.82,overcast clouds,-33.6500,115.3333,Observatory Guest House
10,Tuktoyaktuk,CA,50.00,mist,69.4541,-133.0374,End of The Road Ltd.
14,Hermanus,ZA,50.81,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
43,Narsaq,GL,57.42,clear sky,60.9167,-46.0500,Sleep In Narsaq
45,Iqaluit,CA,53.33,few clouds,63.7506,-68.5145,Frobisher Inn
...,...,...,...,...,...,...,...
646,Anadyr,RU,50.50,light rain,64.7500,177.4833,Отке17.ру — жильё для командированных
647,Altay,CN,58.57,clear sky,47.8667,88.1167,Jinqiao Hotel
651,Usinsk,RU,56.16,scattered clouds,65.9939,57.5281,Dobro Pozhalovaty
658,Hrodna,BY,50.94,scattered clouds,53.6884,23.8258,Гостиница Семашко на Антонова


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.25)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))